# Variational Autoencoder
*Zhiang Chen, March 2017*

Adapted from the blog here: https://jmetzen.github.io/2015-11-27/vae.html

### 1. Import packages

In [16]:
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
import matplotlib.pyplot as plt
import random
import operator
import time
import os
import math
import deepdish as dd
from tensorflow.contrib.learn.python.learn.datasets.mnist import read_data_sets
from math import *

### 2. Import data

In [2]:
wd = os.getcwd()
os.chdir('..')
file_name = 'depth_data.h5'

save = dd.io.load(file_name)

train_objects = save['train_objects']
train_orientations = save['train_orientations']
train_values = save['train_values']
valid_objects = save['valid_objects']
valid_orientations = save['valid_orientations']
valid_values = save['valid_values']
test_objects = save['test_objects']
test_orientations = save['test_orientations']
test_values = save['test_values']
value2object = save['value2object']
object2value = save['object2value']
del save

os.chdir(wd)

print('training dataset', train_objects.shape, train_orientations.shape, train_values.shape)
print('validation dataset', valid_objects.shape, valid_orientations.shape, valid_values.shape)
print('testing dataset', test_objects.shape, test_orientations.shape, test_values.shape)

('training dataset', (427680, 11), (427680, 10), (427680, 40, 40, 1))
('validation dataset', (71226, 11), (71226, 10), (71226, 40, 40, 1))
('testing dataset', (71280, 11), (71280, 10), (71280, 40, 40, 1))


### 3. Shuffle data

In [3]:
image_size = 40

def randomize(dataset, classes, angles):
    permutation = np.random.permutation(classes.shape[0])
    shuffled_dataset = dataset[permutation,:,:]
    shuffled_classes = classes[permutation]
    shuffled_angles = angles[permutation]
    return shuffled_dataset, shuffled_classes, shuffled_angles

train_dataset, train_classes, train_angles = randomize(train_values, train_objects, train_orientations)
valid_dataset, valid_classes, valid_angles = randomize(valid_values, valid_objects, valid_orientations)
test_dataset, test_classes, test_angles = randomize(test_values, test_objects, test_orientations)

valid_dataset = valid_dataset[:5000,:,:,:]
valid_angles = valid_angles[:5000,:]
valid_classes = valid_classes[:5000,:]

test_dataset = test_dataset[:5000,:,:,:]
test_angles = test_angles[:5000,:]
test_classes = test_classes[:5000,:]

train_dataset = train_dataset.reshape(-1,image_size*image_size)
test_dataset = test_dataset.reshape(-1,image_size*image_size)
n_samples = train_dataset.shape[0]

In [2]:
'''MNIST'''
mnist = read_data_sets('MNIST_data', one_hot=True)
n_samples = mnist.train.num_examples
train_dataset = mnist.train.images

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


### 4. Define some functions

In [3]:
np.random.seed(0)
tf.set_random_seed(0)


def xavier_init(fan_in, fan_out, constant=1): 
    """ Xavier initialization of network weights"""
    # https://stackoverflow.com/questions/33640581/how-to-do-xavier-initialization-on-tensorflow
    low = -constant*np.sqrt(6.0/(fan_in + fan_out)) 
    high = constant*np.sqrt(6.0/(fan_in + fan_out))
    return tf.random_uniform((fan_in, fan_out), 
                             minval=low, maxval=high, 
                             dtype=tf.float32)

### 5. VAE

In [36]:
class VariationalAutoencoder(object):
    """ Variation Autoencoder (VAE) with an sklearn-like interface implemented using TensorFlow.
    
    This implementation uses probabilistic encoders and decoders using Gaussian 
    distributions and  realized by multi-layer perceptrons. The VAE can be learned
    end-to-end.
    
    See "Auto-Encoding Variational Bayes" by Kingma and Welling for more details.
    """
    def __init__(self, network_architecture, transfer_fct=tf.nn.softplus, 
                 learning_rate=0.001, batch_size=100):
        self.network_architecture = network_architecture
        self.transfer_fct = transfer_fct
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        
        # tf Graph input
        self.x = tf.placeholder(tf.float32, [batch_size, network_architecture["n_input"]])
        self.image_size = float(network_architecture["n_input"])
        
        # Create autoencoder network
        self._create_network()
        # Define loss function based variational upper-bound and 
        # corresponding optimizer
        self._create_loss_optimizer()
        
        # Initializing the tensor flow variables
        init = tf.global_variables_initializer()

        # Launch the session
        config = tf.ConfigProto()
        config.gpu_options.allow_growth=True
        config.log_device_placement = True
        config.gpu_options.allocator_type = 'BFC' 
        self.sess = tf.InteractiveSession(config = config)
        self.sess.run(init)
    
    def _create_network(self):
        # Initialize autoencode network weights and biases
        network_weights = self._initialize_weights(**self.network_architecture)

        # Use recognition network to determine mean and 
        # (log) variance of Gaussian distribution in latent
        # space
        self.z_mean, self.z_log_sigma_sq = \
            self._recognition_network(network_weights["weights_recog"], 
                                      network_weights["biases_recog"])

        # Draw one sample z from Gaussian distribution
        n_z = self.network_architecture["n_z"]
        eps = tf.random_normal((self.batch_size, n_z), 0, 1, 
                               dtype=tf.float32)
        # z = mu + sigma*epsilon
        self.z = tf.add(self.z_mean, 
                        tf.multiply(tf.sqrt(tf.exp(self.z_log_sigma_sq)), eps))

        # Use generator to determine mean of
        # Bernoulli distribution of reconstructed input
        self.x_reconstr_mean, self.x_reconstr_log_sigma_sq = \
            self._generator_network(network_weights["weights_gener"],
                                    network_weights["biases_gener"])
            
    def _initialize_weights(self, n_hidden_recog_1, n_hidden_recog_2, 
                            n_hidden_gener_1,  n_hidden_gener_2, 
                            n_input, n_z):
        all_weights = dict()
        all_weights['weights_recog'] = {
            'h1': tf.Variable(xavier_init(n_input, n_hidden_recog_1)),
            'h2': tf.Variable(xavier_init(n_hidden_recog_1, n_hidden_recog_2)),
            'out_mean': tf.Variable(xavier_init(n_hidden_recog_2, n_z)),
            'out_log_sigma': tf.Variable(xavier_init(n_hidden_recog_2, n_z))}
        all_weights['biases_recog'] = {
            'b1': tf.Variable(tf.zeros([n_hidden_recog_1], dtype=tf.float32)),
            'b2': tf.Variable(tf.zeros([n_hidden_recog_2], dtype=tf.float32)),
            'out_mean': tf.Variable(tf.zeros([n_z], dtype=tf.float32)),
            'out_log_sigma': tf.Variable(tf.zeros([n_z], dtype=tf.float32))}
        all_weights['weights_gener'] = {
            'h1': tf.Variable(xavier_init(n_z, n_hidden_gener_1)),
            'h2': tf.Variable(xavier_init(n_hidden_gener_1, n_hidden_gener_2)),
            'out_mean': tf.Variable(xavier_init(n_hidden_gener_2, n_input)),
            'out_log_sigma': tf.Variable(xavier_init(n_hidden_gener_2, n_input))}
        all_weights['biases_gener'] = {
            'b1': tf.Variable(tf.zeros([n_hidden_gener_1], dtype=tf.float32)),
            'b2': tf.Variable(tf.zeros([n_hidden_gener_2], dtype=tf.float32)),
            'out_mean': tf.Variable(tf.zeros([n_input], dtype=tf.float32)),
            'out_log_sigma': tf.Variable(tf.zeros([n_input], dtype=tf.float32))}
        return all_weights
            
    def _recognition_network(self, weights, biases):
        # Generate probabilistic encoder (recognition network), which
        # maps inputs onto a normal distribution in latent space.
        # The transformation is parametrized and can be learned.
        layer_1 = self.transfer_fct(tf.add(tf.matmul(self.x, weights['h1']), 
                                           biases['b1'])) 
        layer_2 = self.transfer_fct(tf.add(tf.matmul(layer_1, weights['h2']), 
                                           biases['b2'])) 
        z_mean = tf.add(tf.matmul(layer_2, weights['out_mean']),
                        biases['out_mean'])
        z_log_sigma_sq = \
            tf.add(tf.matmul(layer_2, weights['out_log_sigma']), 
                   biases['out_log_sigma'])
        return (z_mean, z_log_sigma_sq)

    def _generator_network(self, weights, biases):
        # Generate probabilistic decoder (decoder network), which
        # maps points in latent space onto a Bernoulli distribution in data space.
        # The transformation is parametrized and can be learned.
        layer_1 = self.transfer_fct(tf.add(tf.matmul(self.z, weights['h1']), 
                                           biases['b1'])) 
        layer_2 = self.transfer_fct(tf.add(tf.matmul(layer_1, weights['h2']), 
                                           biases['b2'])) 
        x_reconstr_mean = tf.add(tf.matmul(layer_2, weights['out_mean']),biases['out_mean'])
        
        #x_reconstr_mean = tf.sigmoid(tf.add(tf.matmul(layer_2, weights['out_mean']),biases['out_mean']))
            
        x_reconstr_log_sigma_sq = \
            tf.add(tf.matmul(layer_2, weights['out_log_sigma']),
                                 biases['out_log_sigma'])
            
        return x_reconstr_mean, x_reconstr_log_sigma_sq
            
    def _create_loss_optimizer(self):
        '''
        # The loss is composed of two terms:
        # 1.) The reconstruction loss (the negative log probability
        #     of the input under the reconstructed Bernoulli distribution 
        #     induced by the decoder in the data space).
        #     This can be interpreted as the number of "nats" required
        #     for reconstructing the input when the activation in latent
        #     is given.
        # Adding 1e-10 to avoid evaluation of log(0.0)
        self.disp = self.x_reconstr_mean
        self.reconstr_loss = \
            -tf.reduce_sum(self.x * tf.log(1e-10 + self.x_reconstr_mean)
                           + (1-self.x) * tf.log(1e-10 + 1 - self.x_reconstr_mean),
                           1)
        '''
        # The p(x|z) has multi-Gaussian distribution
        sigma_sq = tf.exp(self.x_reconstr_log_sigma_sq)
        std_dev = tf.sqrt(sigma_sq)
        reconstr_p = tf.contrib.distributions.MultivariateNormalDiag(self.x_reconstr_mean, std_dev)
        inverse_sigma_sq = tf.matrix_diag(tf.reciprocal(sigma_sq))
        #scalar = tf.matmul(tf.matmul(tf.transpose(self.x-self.x_reconstr_mean),inverse_sigma_sq),(self.x-self.x_reconstr_mean))
        
        self.reconstr_loss = -tf.log(1e-10+reconstr_p.pdf(self.x))
        
        x_u = tf.reshape(tf.transpose(self.x-self.x_reconstr_mean),[self.batch_size, tf.shape(self.x)[1],1])
        energe = -0.5*tf.matmul(tf.matmul(tf.transpose(x_u, perm=[0, 2, 1]),inverse_sigma_sq), x_u)
        s = tf.reciprocal(tf.sqrt(tf.matrix_determinant(tf.matrix_diag(sigma_sq))))
        p = tf.constant((2*pi)**(-self.image_size/2))
        normalizer = s
        
        print sigma_sq[0,:]
        self.disp = sigma_sq[0,:]
        #print(reconstr_p.get_batch_shape())
        #print(reconstr_p.get_event_shape())
        #'''
        
        # 2.) The latent loss, which is defined as the Kullback Leibler divergence 
        ##    between the distribution in latent space induced by the encoder on 
        #     the data and some prior. This acts as a kind of regularizer.
        #     This can be interpreted as the number of "nats" required
        #     for transmitting the the latent space distribution given
        #     the prior.
        self.latent_loss = -0.5 * tf.reduce_sum(1 + self.z_log_sigma_sq 
                                           - tf.square(self.z_mean) 
                                           - tf.exp(self.z_log_sigma_sq), 1)
        self.cost = tf.reduce_mean(self.reconstr_loss + self.latent_loss)   # average over batch
        # Use ADAM optimizer
        self.optimizer = \
            tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(self.cost)
        
    def partial_fit(self, X):
        """Train model based on mini-batch of input data.
        
        Return cost of mini-batch.
        """
        opt, cost,l,r,disp = self.sess.run((self.optimizer, self.cost, self.latent_loss, self.reconstr_loss, self.disp), 
                                  feed_dict={self.x: X})
        return cost,l,r,disp
    
    def transform(self, X):
        """Transform data by mapping it into the latent space."""
        # Note: This maps to mean of distribution, we could alternatively
        # sample from Gaussian distribution
        return self.sess.run(self.z_mean, feed_dict={self.x: X})
    
    def generate(self, z_mu=None):
        """ Generate data by sampling from latent space.
        
        If z_mu is not None, data for this point in latent space is
        generated. Otherwise, z_mu is drawn from prior in latent 
        space.        
        """
        if z_mu is None:
            z_mu = np.random.normal(size=self.network_architecture["n_z"])
        # Note: This maps to mean of distribution, we could alternatively
        # sample from Gaussian distribution
        return self.sess.run(self.x_reconstr_mean, 
                             feed_dict={self.z: z_mu})
    
    def reconstruct(self, X):
        """ Use VAE to reconstruct given data. """
        return self.sess.run(self.x_reconstr_mean, 
                             feed_dict={self.x: X})

### 6. Train 

In [37]:
def train(network_architecture, learning_rate=0.001, transfer_fct=tf.nn.softplus,
          batch_size=100, training_epochs=100, display_step=20):
    vae = VariationalAutoencoder(network_architecture, 
                                 learning_rate=learning_rate, 
                                 batch_size=batch_size,
                                 transfer_fct=tf.nn.softplus)
    
    sum_cost = 0.0
    for step in range(training_epochs):
        offset = (step * batch_size) % (train_dataset.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        cost,l,r,disp = vae.partial_fit(batch_data)
        sum_cost += cost
        if ((step % display_step == 0) & (step!=0)):
            avg_step = sum_cost/step
            print("Epoch:", '%d' % (step), "cost", "{:.9f}".format(avg_step))
            print('latent_loss:')
            print(l)
            print('reconstr_loss')
            print(r)
            print('disp')
            print(disp)
            
    return vae

network_architecture = \
    dict(n_hidden_recog_1=1000, # 1st layer encoder neurons
         n_hidden_recog_2=500, # 2nd layer encoder neurons
         n_hidden_gener_1=500, # 1st layer decoder neurons
         n_hidden_gener_2=1000, # 2nd layer decoder neurons
         n_input=28*28, # MNIST data input (img shape: 28*28)
         n_z=20)  # dimensionality of latent space

vae = train(network_architecture, training_epochs=20000, learning_rate=0.001, transfer_fct=tf.nn.softplus)

Tensor("strided_slice_15:0", shape=(784,), dtype=float32)
('Epoch:', '20', 'cost', '28.946263027')
latent_loss:
[ 0.53423083  0.56153119  0.49477789  0.52286398  0.54887748  0.58194822
  0.49725628  0.52852809  0.70528102  0.50914663  0.54700601  0.48735878
  0.49582055  0.46773821  0.52785158  0.50089711  0.46263644  0.53390533
  0.48505658  0.45232892  0.49491671  0.5308575   0.53437519  0.50982392
  0.50953197  0.55981255  0.48890838  0.5134275   0.52018821  0.49800813
  0.54097009  0.52017736  0.46686879  0.50900853  0.54870844  0.51936209
  0.48227972  0.52016705  0.62236685  0.5699631   0.5065285   0.47114313
  0.53471529  0.4585453   0.53601122  0.44192076  0.63972855  0.52239805
  0.53770924  0.55516434  0.50524259  0.5052352   0.59945911  0.47508726
  0.55262017  0.50247592  0.50859547  0.51506102  0.49597043  0.48381329
  0.50146019  0.46316636  0.56508231  0.48723429  0.52209067  0.4962545
  0.50982273  0.48732674  0.5661484   0.4992269   0.52476645  0.45819545
  0.54657495 

KeyboardInterrupt: 